In [1]:
import cv2
import os
import numpy as np

# Building Face Classifier

In [3]:
def distance (v1,v2):
    return np.sqrt(((v1-v2)**2).sum())

def knn(train,test,k=5):
    dist=[]
    for i in range(train.shape[0]):
        ix=train[i,:-1]
        iy=train[i,-1]
        d=distance(test,ix)
        dist.append([d,iy])
    dk=sorted(dist,key=lambda x:x[0])[:k]
    labels=np.array(dk)[:,-1]
    output=np.unique(labels,return_counts=True)
    index=np.argmax(output[1])
    return output[0][index]

In [4]:
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
data_set_path='I:/Face Recognition/'
face_data=[]
labels=[]
class_id=0
names={}

In [5]:
for fx in os.listdir(data_set_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]
        data_item=np.load(data_set_path+fx)
        face_data.append(data_item)
        target=class_id*np.ones((data_item.shape[0]))
        class_id+=1
        labels.append(target)
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
trainset=np.concatenate((face_dataset,face_labels),axis=1)
face_dataset.shape

(42, 30000)

# testing

In [6]:
cap=cv2.VideoCapture(0)
count=0
while True:
    ret,frame=cap.read()
    if ret==False:
        print("no face found!!!")
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    for (x,y,w,h) in faces:
        offset=10
        offset=10
        face_section=frame[y-offset:y+offset+h,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100))
        out=knn(trainset,face_section.flatten())
        pred_name=names[int(out)]
        count+=1
        #print(pred_name)
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_TRIPLEX,1,(255,0,0),1,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()